## Prepare a line network

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ashusri_0000051121
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion-2.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ashusri_0000051121
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion-2.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 /home/fabric/work/fabric_config/fabric_bastion_key
!chmod 600 /home/fabric/work/fabric_config/slice_key

### Get resources

In [3]:
slice_name="line-fig-9-slice_7"

if fablib.get_slice(slice_name):
    print("You already have a slice named %s - skip the next part." % slice_name)
    slice = fablib.get_slice(name=slice_name)
    print(slice)

You already have a slice named line-fig-9-slice_1 - skip the next part.
-----------  ------------------------------------
Slice Name   line-fig-9-slice_1
Slice ID     cae828d3-6676-416b-8629-f3a1acb19337
Slice State  StableOK
Lease End    2023-01-06 23:30:19 +0000
-----------  ------------------------------------


In [4]:
slice = fablib.new_slice(name=slice_name)

In [5]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         6  174/192  1484/1536  60534/60600    368/381                 2/2                             2/2                            10/10               2/2               3/3
SALT         6  176/192  1504/1536  60520/60600    375/381                 2/2                             2/2                            10/10               2/2               3/3
NCSA         6  152/192  1376/1536  60100/60600    372/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  242/320  2312/2560  115730/116400  620/635                 2/2

In [20]:
site_name = fablib.get_random_site()
site_name

'MASS'

In [21]:
slice = fablib.new_slice(name=slice_name)

In [22]:
node_names = ["client", "emulator", "router", "server"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_20')

In [23]:
nets = [
    {"name": "client-em",   "nodes": ["client", "emulator"]},
    {"name": "em-rtr",      "nodes": ["emulator", "router"]},
    {"name": "rtr-server",  "nodes": ["router", "server"]},
]

In [24]:
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]+"_" + node).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

In [25]:
slice.submit()


-----------  ------------------------------------
Slice Name   line-fig-9-slice_1
Slice ID     cae828d3-6676-416b-8629-f3a1acb19337
Slice State  StableOK
Lease End    2023-01-03 19:12:37 +0000
-----------  ------------------------------------

Retry: 22, Time: 273 sec

ID                                    Name      Site    Host                          Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  --------  ------  --------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
ac853e65-ebf9-4a6e-8019-8defe43fd4cb  client    MASS    mass-w3.fabric-testbed.net        4     32     100  default_ubuntu_20  205.172.170.84   Active
67c40946-1321-4394-9e17-1bd10f4d9608  emulator  MASS    mass-w3.fabric-testbed.net        4     32     100  default_ubuntu_20  205.172.170.73   Active
70cf4c80-a25b-41b0-92c8-5d1a191a359b  router    MASS    mass-w3.fabric-testbed.net        4     32     100 

'cae828d3-6676-416b-8629-f3a1acb19337'

In [26]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [4]:
for node in slice.get_nodes():
    print(f"{node}")

-----------------  -------------------------------------------------------------------------------------------------------------------------
ID                 ac853e65-ebf9-4a6e-8019-8defe43fd4cb
Name               client
Cores              4
RAM                32
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               mass-w3.fabric-testbed.net
Site               MASS
Management IP      205.172.170.84
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J ashusri_0000051121@bastion-1.fabric-testbed.net ubuntu@205.172.170.84
-----------------  -------------------------------------------------------------------------------------------------------------------------
-----------------  -------------------------------------------------------------------------------------------------------------------------
ID                 67c40946-1321-4394-9e17-1bd10f4d9608
Name               emulato

### Set up the line topology

### Configure addresses

In [5]:
if_conf = {
    "client-client-em_client-p1":     {"addr": "10.0.1.1", "subnet": "10.0.1.0/24"},
    "emulator-client-em_emulator-p1": {"addr": "10.0.1.2", "subnet": "10.0.1.0/24"},
    "emulator-em-rtr_emulator-p1":    {"addr": "10.0.2.1", "subnet": "10.0.2.0/24"},
    "router-em-rtr_router-p1":        {"addr": "10.0.2.2", "subnet": "10.0.2.0/24"},
    "router-rtr-server_router-p1":    {"addr": "10.0.3.2", "subnet": "10.0.3.0/24"},
    "server-rtr-server_server-p1":    {"addr": "10.0.3.1", "subnet": "10.0.3.0/24"}
}

In [6]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))

In [7]:
rt_conf = [
    {"name": "client",   "addr": "10.0.3.0/24", "gw": "10.0.1.2"},
    {"name": "emulator", "addr": "10.0.3.0/24", "gw": "10.0.2.2"},
    {"name": "router",   "addr": "10.0.1.0/24", "gw": "10.0.2.1"},
    {"name": "server",   "addr": "10.0.1.0/24", "gw": "10.0.3.2"}
]

In [8]:
for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [9]:
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [10]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

### Install stuff

In [11]:
apt_inst = [
    {"name": "client",  "packages": ["iperf3", "moreutils"]},
    {"name": "router",  "packages": ["moreutils","jq"]},
    {"name": "server",  "packages": ["iperf3", "moreutils"]},
]

In [12]:
for a in apt_inst:
    slice.get_node(name=a['name']).execute("sudo apt update")
    slice.get_node(name=a['name']).execute("sudo apt -y install " + " ".join(a['packages']))

In [4]:
import datetime
from datetime import timedelta

now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")

slice.renew(end_date)

### Experiment configuration

In [5]:
btl_capacity = 100 # mbit
base_owd     = 20  # ms, to be applied in each direction

In [6]:
slice.get_node(name="client").execute("sudo modprobe tcp_bbr")

('', '')

### configure delay at emulator

In [16]:
em = [
    {"name": "emulator", "if": "emulator-client-em_emulator-p1"},
    {"name": "emulator", "if": "emulator-em-rtr_emulator-p1"}
]

for e in em:
    os_if_name = slice.get_interface(name=e['if']).get_os_interface()
    cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=os_if_name, owd=base_owd)
    slice.get_node(name=e["name"]).execute(cmds)

### Iterate over bottleneck buffer size (in multiples of BDP) and number of BBR flows

In [ ]:
%%capture output
import time


N_bdp = [0.5, 5, 10, 20, 30, 40, 50]

#N_bdp = [0.5, 5, 10]

d = 120

trials = 1


for i, n_bdp in enumerate(N_bdp):
    
    btl_limit    = int(1000*n_bdp*btl_capacity*2*base_owd/8)
    
    print("Buffer size = {num_bdp} times the BDP".format(num_bdp=n_bdp));
        
    ## configure network
    
    btl = [
        {"name": "router", "if": "router-rtr-server_router-p1"}
    ]    
    for b in btl:
        os_if_name = slice.get_interface(name=b['if']).get_os_interface()
        cmds = '''
        sudo tc qdisc del dev {iface} root
        sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
        sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
        sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer} 
        '''.format(iface=os_if_name, capacity=btl_capacity, buffer=btl_limit)
        slice.get_node(name=b["name"]).execute(cmds)
    
    ## iterate over number of BBR flows
    
    for j in range(0, 51):
        
        print("Staring {num_trials} trials of experiment with {flows} BBR flows, each for {duration} seconds".format(flows=j, duration=d, num_trials=trials))
        
        ## carry out multiple trials of each experiment
        
        sleep_time = 60*10
        k=1

        while k<= trials:
            try:

                # start BBR and non-BBR server instance
                slice.get_node(name="server").execute("iperf3 -s -1 -p 4000 -D")
                slice.get_node(name="server").execute("iperf3 -s -1 -p 5000 -D")
                slice.get_node(name="client").execute_thread("sleep 1; iperf3 -c 10.0.3.1 -t {duration} -P {flows} -C bbr -p 4000 -J > bbr-result-{num_bdp}-{num_bbr}-{trial_no}.json".format(duration=d, flows=j, num_bdp = n_bdp, num_bbr=j, trial_no = k))
                stdout, stderr = slice.get_node(name="client").execute("sleep 1; iperf3 -c 10.0.3.1 -t {duration} -P {flows} -C cubic -p 5000 -i 0.1 -J > cubic-result-{num_bdp}-{num_bbr}-{trial_no}.json".format(duration=d, flows=50-j, num_bdp = n_bdp, num_bbr=j, trial_no = k))
                time.sleep(5)
                k=k+1
            except:
                # repeat this attempt since it was not succesful, don't increment k

                print("Something went wrong at time %f, waiting %d seconds then retrying with k=%d" % (time.time(), sleep_time, k))
                time.sleep(sleep_time)



slice.get_node(name="client").execute("mkdir figure_9_results; mv *.json ./figure_9_results")

slice.get_node(name="client").execute("sudo apt update; sudo apt -y install jq")

# upload_file(local_file_path: str, remote_file_path: str, retry: int = 3, retry_interval: int = 10)

# upload data processing files

local_file_path_bbr = "/home/fabric/work/cnert_bbr_project/figure_9_parallel/extract_bbr_values.sh"
local_file_path_cubic = "/home/fabric/work/cnert_bbr_project/figure_9_parallel/extract_cubic_values.sh"

remote_file_path_bbr = "/home/ubuntu/extract_bbr_values.sh"
remote_file_path_cubic = "/home/ubuntu/extract_cubic_values.sh"

slice.get_node(name="client").upload_file(local_file_path_bbr, remote_file_path_bbr)


slice.get_node(name="client").upload_file(local_file_path_cubic, remote_file_path_cubic)


## do data processing

slice.get_node(name="client").execute("chmod +x extract_cubic_values.sh")
slice.get_node(name="client").execute("chmod +x extract_bbr_values.sh")

slice.get_node(name="client").execute("./extract_cubic_values.sh ./figure_9_results")
slice.get_node(name="client").execute("./extract_bbr_values.sh ./figure_9_results")


## download processed data as csv files

local_file_path_bbr = "/home/fabric/work/cnert_bbr_project/figure_9_parallel/slice_7_results/bbr_results.csv"
local_file_path_cubic = "/home/fabric/work/cnert_bbr_project/figure_9_parallel/slice_7_results/cubic_results.csv"


remote_file_path_bbr = "/home/ubuntu/figure_9_results/bbr_results.csv"
remote_file_path_cubic = "/home/ubuntu/figure_9_results/cubic_results.csv"


slice.get_node(name="client").download_file(local_file_path_bbr, remote_file_path_bbr)


slice.get_node(name="client").download_file(local_file_path_cubic, remote_file_path_cubic)






In [ ]:
output.show()

### Re-run experiments with missing data

In [ ]:
import time
import csv

d = 120

# open the CSV file in read mode
with open('slice_7_results/bbr_results.csv', 'r') as file:  # also do this for bbr_results.csv
    # create a CSV reader
    reader = csv.reader(file)
    
    # get the column names from the first row
    headers = next(reader)
    
    # find the index of the last column
    last_column_index = len(headers) - 1
    
    # loop through the rows
    for row in reader:
        # check if the value in the last column is None
        if row[last_column_index] == 'null':
            # get the N_bdp and flow values
            n_bdp = float(row[headers.index('N_bdp')])
            flow = int(row[headers.index('flow')])
            trial = int(row[headers.index('trial')])
            
            print(f'N_bdp: {n_bdp}, flow: {flow}, trial: {trial}')
            
            
            btl_limit    = int(1000*n_bdp*btl_capacity*2*base_owd/8)
    
            print("Buffer size = {num_bdp} times the BDP".format(num_bdp=n_bdp));
        
            ## configure network
    
            btl = [
                {"name": "router", "if": "router-rtr-server_router-p1"}
            ]    
            for b in btl:
                os_if_name = slice.get_interface(name=b['if']).get_os_interface()
                cmds = '''
                sudo tc qdisc del dev {iface} root
                sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
                sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
                sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer} 
                '''.format(iface=os_if_name, capacity=btl_capacity, buffer=btl_limit)
                slice.get_node(name=b["name"]).execute(cmds)
                
                # start BBR and non-BBR server instance
                slice.get_node(name="server").execute("iperf3 -s -1 -p 4000 -D")
                slice.get_node(name="server").execute("iperf3 -s -1 -p 5000 -D")
                #slice.get_node(name="router").execute_thread(router_script.format(expid=i, iface=os_if_name, duration=d))
                slice.get_node(name="client").execute_thread("sleep 1; iperf3 -c 10.0.3.1 -t {duration} -P {flows} -C bbr -p 4000 -J > bbr-result-{num_bdp}-{num_bbr}-{trial_no}.json".format(duration=d, flows=flow, num_bdp = n_bdp, num_bbr=flow, trial_no = trial))
                stdout, stderr = slice.get_node(name="client").execute("sleep 1; iperf3 -c 10.0.3.1 -t {duration} -P {flows} -C cubic -p 5000 -i 0.1 -J > cubic-result-{num_bdp}-{num_bbr}-{trial_no}.json".format(duration=d, flows=50-flow, num_bdp = n_bdp, num_bbr=flow, trial_no = trial))
                time.sleep(5)

